In [1]:
import numpy as np
import metpy.calc as mpcalc
from metpy.units import units
from nzthermo.functional import Intersection
import nzthermo as nzt
import nzthermo.functional as F
from nzthermo.core import ParcelProfile

pascal = units.pascal
kelvin = units.kelvin


PRESSURE = np.array(
    [1013, 1000, 975, 950, 925, 900, 875, 850, 825, 800, 775, 750, 725, 700, 650, 600, 550, 500, 450, 400, 350, 300],
    dtype=np.float64
)
PRESSURE *= 100.0
TEMPERATURE = np.array(
    [
        [243, 242, 241, 240, 239, 237, 236, 235, 233, 232, 231, 229, 228, 226, 235, 236, 234, 231, 226, 221, 217, 211],
        [250, 249, 248, 247, 246, 244, 243, 242, 240, 239, 238, 236, 235, 233, 240, 239, 236, 232, 227, 223, 217, 211],
        [293, 292, 290, 288, 287, 285, 284, 282, 281, 279, 279, 280, 279, 278, 275, 270, 268, 264, 260, 254, 246, 237],
        [300, 299, 297, 295, 293, 291, 292, 291, 291, 289, 288, 286, 285, 285, 281, 278, 273, 268, 264, 258, 251, 242],
    ],
    dtype=np.float64
)
DEWPOINT = np.array(
    [
        [224, 224, 224, 224, 224, 223, 223, 223, 223, 222, 222, 222, 221, 221, 233, 233, 231, 228, 223, 218, 213, 207],
        [233, 233, 232, 232, 232, 232, 231, 231, 231, 231, 230, 230, 230, 229, 237, 236, 233, 229, 223, 219, 213, 207],
        [288, 288, 287, 286, 281, 280, 279, 277, 276, 275, 270, 258, 244, 247, 243, 254, 262, 248, 229, 232, 229, 224],
        [294, 294, 293, 292, 291, 289, 285, 282, 280, 280, 281, 281, 278, 274, 273, 269, 259, 246, 240, 241, 226, 219],
    ],
    dtype=np.float64
)

data = {
    "pressure_top":[],
    "temperature_top":[],
    "pressure_bottom":[],
    "temperature_bottom":[],
}

for i in range(DEWPOINT.shape[0]):
    prof = mpcalc.parcel_profile(PRESSURE * pascal, TEMPERATURE[i, 0] * kelvin, DEWPOINT[i, 0] * kelvin)
    lfc_p, lfc_t = mpcalc.lfc(PRESSURE * pascal, TEMPERATURE[i] * kelvin, DEWPOINT[i] * kelvin, prof, which="top")
    data['pressure_top'].append(lfc_p.m)
    data['temperature_top'].append(lfc_t.m)
    lfc_p, lfc_t = mpcalc.lfc(PRESSURE * pascal, TEMPERATURE[i] * kelvin, DEWPOINT[i] * kelvin, prof, which="bottom")
    data['pressure_bottom'].append(lfc_p.m)
    data['temperature_bottom'].append(lfc_t.m)

data

{'pressure_top': [nan, nan, 87345.13814747725, 51262.93216313996],
 'temperature_top': [nan, nan, 283.8777807314008, 269.3086175947326],
 'pressure_bottom': [nan, nan, 90251.30476144128, 91193.02729740732],
 'temperature_bottom': [nan, nan, 285.2035392967315, 291.96125998473383]}

In [2]:
def _less_or_close(a, value, **kwargs):
    r"""Compare values for less or close to boolean masks.

    Returns a boolean mask for values less than or equal to a target within a specified
    absolute or relative tolerance (as in :func:`numpy.isclose`).

    Parameters
    ----------
    a : array-like
        Array of values to be compared
    value : float
        Comparison value

    Returns
    -------
    array-like
        Boolean array where values are less than or nearly equal to value

    """
    return (a < value) | np.isclose(a, value, **kwargs)
def _greater_or_close(a, value, **kwargs):
    r"""Compare values for greater or close to boolean masks.

    Returns a boolean mask for values greater than or equal to a target within a specified
    absolute or relative tolerance (as in :func:`numpy.isclose`).

    Parameters
    ----------
    a : array-like
        Array of values to be compared
    value : float
        Comparison value

    Returns
    -------
    array-like
        Boolean array where values are greater than or nearly equal to value.

    """
    return (a > value) | np.isclose(a, value, **kwargs)


In [3]:
pp = nzt.parcel_profile(PRESSURE, TEMPERATURE, DEWPOINT)

for which in ("top", "bottom"):
    lfc_p, lfc_t = pp.lfc(which)
    for i in range(TEMPERATURE.shape[0]):
        lfc_p_, lfc_t_ = mpcalc.lfc(PRESSURE * pascal, TEMPERATURE[i] * kelvin, DEWPOINT[i] * kelvin, which=which)
        np.testing.assert_allclose(lfc_p[i], lfc_p_.m, rtol=1e-2)
        np.testing.assert_allclose(lfc_t[i], lfc_t_.m, rtol=1e-2)



In [17]:
from nzthermo.const import Rd
PRESSURE = np.array(
    [1013, 1000, 975, 950, 925, 900, 875, 850, 825, 800, 775, 750, 725, 700, 650, 600, 550, 500, 450, 400, 350, 300],
    dtype=np.float64
)
PRESSURE *= 100.0
TEMPERATURE = np.array(
    [
        [243, 242, 241, 240, 239, 237, 236, 235, 233, 232, 231, 229, 228, 226, 235, 236, 234, 231, 226, 221, 217, 211],
        [250, 249, 248, 247, 246, 244, 243, 242, 240, 239, 238, 236, 235, 233, 240, 239, 236, 232, 227, 223, 217, 211],
        [293, 292, 290, 288, 287, 285, 284, 282, 281, 279, 279, 280, 279, 278, 275, 270, 268, 264, 260, 254, 246, 237],
        [300, 299, 297, 295, 293, 291, 292, 291, 291, 289, 288, 286, 285, 285, 281, 278, 273, 268, 264, 258, 251, 242],
    ],
    dtype=np.float64
)
DEWPOINT = np.array(
    [
        [224, 224, 224, 224, 224, 223, 223, 223, 223, 222, 222, 222, 221, 221, 233, 233, 231, 228, 223, 218, 213, 207],
        [233, 233, 232, 232, 232, 232, 231, 231, 231, 231, 230, 230, 230, 229, 237, 236, 233, 229, 223, 219, 213, 207],
        [288, 288, 287, 286, 281, 280, 279, 277, 276, 275, 270, 258, 244, 247, 243, 254, 262, 248, 229, 232, 229, 224],
        [294, 294, 293, 292, 291, 289, 285, 282, 280, 280, 281, 281, 278, 274, 273, 269, 259, 246, 240, 241, 226, 219],
    ],
    dtype=np.float64
)


mp_cape = []
mp_cin = []
for i in range(TEMPERATURE.shape[0]):

    prof = mpcalc.parcel_profile(PRESSURE * pascal, TEMPERATURE[i, 0] * kelvin, DEWPOINT[i, 0] * kelvin)
    cape, cin = mpcalc.cape_cin(PRESSURE * pascal, TEMPERATURE[i] * kelvin, DEWPOINT[i] * kelvin, prof, "bottom", 'top')
    mp_cape.append(cape.m)
    mp_cin.append(cin.m)



print(f"""
[CAPE CIN]
{mp_cape}
{mp_cin}
""")
# .................................................................................................
self  = nzt.parcel_profile(PRESSURE, TEMPERATURE, DEWPOINT)
pressure, temperature, dewpoint, parcel_profile = self.without_lcl() #PRESSURE, TEMPERATURE, DEWPOINT, nzt.parcel_profile(PRESSURE, TEMPERATURE, DEWPOINT)
# pp = parcel_profile
# self = parcel_profile
lcl_p = pressure_lcl = self.lcl_pressure
below_lcl = pressure > pressure_lcl[:, np.newaxis]
# The mixing ratio of the parcel comes from the dewpoint below the LCL, is saturated
# based on the temperature above the LCL

parcel_mixing_ratio = np.where(
    below_lcl,
    nzt.saturation_mixing_ratio(pressure, dewpoint),
    nzt.saturation_mixing_ratio(pressure, temperature)
)

# Convert the temperature/parcel profile to virtual temperature
temperature = nzt.virtual_temperature(temperature, nzt.saturation_mixing_ratio(pressure, dewpoint))
parcel_profile = nzt.virtual_temperature(parcel_profile, parcel_mixing_ratio)
# Calculate LFC limit of integration
lfc_p, _ = self.lfc('bottom')
# Calculate the EL limit of integration
el_p, _ = self.el('top')
# No EL and we use the top reading of the sounding.
# el_p = np.where(np.isnan(el_p), pressure[:, -1], el_p)



# Difference between the parcel path and measured temperature profiles
y = (parcel_profile - temperature)
print(y)
x, y = F.zero_crossing(pressure.copy(), y, log_x=True) # (N, Z)

lfc_p = lfc_p.reshape(-1, 1)
el_p = el_p.reshape(-1, 1)
def _less_or_close(a, value, **kwargs):
    return (a < value) | np.isclose(a, value, **kwargs)

def _greater_or_close(a, value, **kwargs):
    return (a >= value) | np.isclose(a, value, **kwargs)

p_mask = _less_or_close(x, lfc_p) & _greater_or_close(x, el_p)
x_clipped, y_clipped = x.copy(), y.copy()
x_clipped[~p_mask] = np.nan
y_clipped[~p_mask] = np.nan


cape = Rd * F.nantrapz(y_clipped, np.log(x_clipped), axis=1)
p_mask = _greater_or_close(x, lfc_p)
x_clipped, y_clipped = x.copy(), y.copy()
x_clipped[~p_mask] = np.nan
y_clipped[~p_mask] = np.nan
cin = Rd * F.nantrapz(y_clipped, np.log(x_clipped), axis=1)

print(f"[CAPE CIN]\n{cape}\n{cin}")


# x   = x[-1]
# y = y[-1]
# mask =  np.isnan(x)
# x, y = x[~mask], y[~mask]
# p_mask = _less_or_close(x, lfc_p.squeeze()[-1]) & _greater_or_close(x, el_p.squeeze()[-1])
# x_clipped, y_clipped = x[p_mask], y[p_mask]
# Rd * np.trapz(y_clipped[::-1], np.log(x_clipped[::-1]))



# print(
#     "\n"
#     '[lfc_p top-delta]',
#     self.lfc('top')[0].item() - mpcalc.lfc(PRESSURE * pascal, TEMPERATURE[0] * kelvin, DEWPOINT[0] * kelvin, which='top')[0].m,
#     '[lfc_p bottom-delta]',
#     self.lfc('bottom')[0].item() - mpcalc.lfc(PRESSURE * pascal, TEMPERATURE[0] * kelvin, DEWPOINT[0] * kelvin, which='bottom')[0].m,
#     '[el_p top-delta]',
#     self.el('top')[0].item() - mpcalc.el(PRESSURE * pascal, TEMPERATURE[0] * kelvin, DEWPOINT[0] * kelvin, which='top')[0].m,
#     '[el_p bottom-delta]',
#     self.el('bottom')[0].item() - mpcalc.el(PRESSURE * pascal, TEMPERATURE[0] * kelvin, DEWPOINT[0] * kelvin, which='bottom')[0].m,
#     sep='\n'
# )


[CAPE CIN]
[0, 0, 40.44955615512096, 27.919841544451284]
[0, 0, -5.231088794616427, -124.6632608849467]

[[  0.           0.10490035  -0.64010527  -1.4173716   -2.22836821
   -2.07463757  -2.95794656  -3.88009459  -3.84303276  -4.84894619
   -5.90016281  -5.99917729  -7.1389513   -7.32514311 -20.87836795
  -26.6973267  -29.8251046  -32.30383423 -33.1932637  -34.57251956
  -37.5500425  -39.282237  ]
 [  0.           0.07911831  -0.71618648  -1.54468367  -2.40788602
   -2.30734722  -3.24493091  -4.22249268  -4.24198408  -5.30574826
   -6.4160661   -6.54218394  -7.71100284  -7.9426192  -19.5957491
  -23.5388204  -25.80438495 -27.43669532 -28.49393827 -31.05959859
  -32.24367927 -34.20413624]
 [  0.          -0.07978896  -0.18444719  -0.32818366  -0.19971671
    0.56961205   0.4107716    1.18349757   0.93028644   1.53837758
    0.48355413  -1.40896426  -1.76385625  -2.41011071  -2.95702238
   -2.21391003  -4.87817878  -5.72185508  -7.55521968  -8.59513926
   -8.83656566  -9.46635209]
 [  

/home/coder/nzthermo/.venv/lib/python3.11/site-packages/nzthermo/functional.py:449: UserWarning: currently there is no implmentation for decreasing.... TODO:...
  warnings.warn("currently there is no implmentation for decreasing.... TODO:...")


In [5]:
import metpy.constants as mpconst

mpconst.Rd, Rd

(287.04749097718457 <Unit('joule / kelvin / kilogram')>, 287.04749097718457)

In [6]:
print("""\
[ 30000.          35000.          40000.          45000.
  50000.          55000.          60000.          65000.
  70000.          72500.          75000.          76850.51450917
  77500.          80000.          82500.          85000.
  87500.          90000.          91851.42562588  92500.
  95000.          97500.         100000.         101300.        ]
  
[ 30000.          35000.          40000.          45000.
   50000.          55000.          60000.          65000.
   70000.          72500.          75000.          76853.41182469
   77500.          80000.          82500.          85000.
   87500.          90000.          91844.40614874  92500.
   95000.          97500.         100000.         101300. nan             nan]

      

[ 30000.          35000.          40000.          45000.
  45342.21426759  50000.          55000.          55511.0838548
  60000.          65000.          70000.          72500.
  75000.          77500.          80000.          82500.
  85000.          87500.          88481.8649275   90000.
  91984.0007076   92500.          95000.          97500.
 100000.         101300.        ]
      
 [ 30000.          35000.          40000.          45000.
   45348.18988747  50000.          55000.          55503.21680679
   60000.          65000.          70000.          72500.
   75000.          77500.          80000.          82500.
   85000.          87500.          88486.6306099   90000.
   91977.69836503  92500.          95000.          97500.
  100000.         101300.        ]
      
""")

[ 30000.          35000.          40000.          45000.
  50000.          55000.          60000.          65000.
  70000.          72500.          75000.          76850.51450917
  77500.          80000.          82500.          85000.
  87500.          90000.          91851.42562588  92500.
  95000.          97500.         100000.         101300.        ]
  
[ 30000.          35000.          40000.          45000.
   50000.          55000.          60000.          65000.
   70000.          72500.          75000.          76853.41182469
   77500.          80000.          82500.          85000.
   87500.          90000.          91844.40614874  92500.
   95000.          97500.         100000.         101300. nan             nan]

      

[ 30000.          35000.          40000.          45000.
  45342.21426759  50000.          55000.          55511.0838548
  60000.          65000.          70000.          72500.
  75000.          77500.          80000.          82500.
  85000.          

In [7]:
def nantrapz(y, x, dtype):
    dcape = np.zeros(x.shape[0], dtype=dtype)  # (N,)
    batch, levels = np.nonzero(np.diff(pressure > p_top[:, np.newaxis], prepend=True, append=False))
    for lvl in np.unique(levels):
        above = levels == lvl
        nx = batch[above]  # the sample indices
        # zx = np.s_[: np.argmin(~np.isnan(pressure[above]).any(axis=0))]
        dcape[nx] = -(Rd * np.trapz(y, x, axis=1))


In [8]:
pressure, temperature, dewpoint, parcel_profile = PRESSURE, TEMPERATURE, DEWPOINT, nzt.parcel_profile(PRESSURE, TEMPERATURE, DEWPOINT)

parcel_profile.without_lcl()
# parcel_profile.profile.shape, parcel_profile.lcl_index

# parcel_profile.profile

# parcel_profile.profile[np.arange(parcel_profile.profile.shape[1])[np.newaxis] != parcel_profile.lcl_index[1][:, np.newaxis]]

(array([[101300., 100000.,  97500.,  95000.,  92500.,  90000.,  87500.,
          85000.,  82500.,  80000.,  77500.,  75000.,  72500.,  70000.,
          65000.,  60000.,  55000.,  50000.,  45000.,  40000.,  35000.,
          30000.]]),
 array([[300., 299., 297., 295., 293., 291., 292., 291., 291., 289., 288.,
         286., 285., 285., 281., 278., 273., 268., 264., 258., 251., 242.]]),
 array([[294., 294., 293., 292., 291., 289., 285., 282., 280., 280., 281.,
         281., 278., 274., 273., 269., 259., 246., 240., 241., 226., 219.]]),
 array([[300.        , 298.89493527, 296.74063224, 294.54650361,
         292.46591672, 291.4894354 , 290.47856949, 289.43071056,
         288.34294526, 287.21200709, 286.0342188 , 284.80542292,
         283.52089783, 282.17525558, 279.27501753, 276.04273398,
         272.3941572 , 268.21213079, 263.33062945, 257.51241765,
         250.42454691, 241.63241352]]))